## Import libraries

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
pd.set_option('display.float_format', '{:.3f}'.format)

## Helper function

In [2]:
def getQoF(mod, X_test, y_test, key, lam=0.0):
    
    y_pred = mod.predict(X_test)
    
    if key == "sqrt":
        y_test = y_test**2
        y_pred = y_pred**2
    elif key == "log1p":
        y_test = np.expm1(y_test)
        y_pred = np.expm1(y_pred)
    elif key == "boxcox":
        y_test = inv_boxcox(y_test, lam)
        y_pred = inv_boxcox(y_pred, lam)
    
    m = len(y_test)
    dfr = mod.df_model
    df = m - (int(dfr) + 1) 
    
    e = y_test - y_pred
    sse = np.sum(e**2)
    sst = np.sum((y_test - np.mean(y_test))**2)
    ssr = sst - sse
    rSq = 1 - sse/sst # R-squared
    #rSqBar = 1 - (1-rSq) * df # adjusted R-squared
    rSqBar = 1 - (1 - rSq) * (m - 1) / df

    msr = ssr / dfr
    mse = sse / m-dfr
    #rmse = np.sqrt(mse)
    rmse = np.sqrt(sse / m)
    
    mae = np.mean(np.abs(e))
    fStat  = msr / mse # F statistic
    
    return {
        "rSq": rSq, "rSqBar": rSqBar, "sst": sst, "sse": sse,
        "mse": mse, "rmse": rmse, "mae": mae,
        "m": m, "dfr": dfr, "df": df, "fStat": fStat
    } 

In [3]:
def fit(X_full, y_full, X_train, y_train):

    # in-sample evaluation using the whole dataset
    model_insample = sm.OLS(y_full, X_full).fit()
    
    # validation using 80% training and 20% testing split
    model_val = sm.OLS(y_train, X_train).fit()
    
    # forward selection
    forward = SequentialFeatureSelector(LinearRegression(), 
                                        n_features_to_select="auto", 
                                        direction="forward", 
                                        tol=0.01)
    
    forward.fit(X_train, y_train)
    forward_cols = list(X_train.columns[forward.get_support()])
    model_forward = sm.OLS(y_train, X_train[forward_cols]).fit()

    # backward selection
    backward = SequentialFeatureSelector(LinearRegression(), 
                                         n_features_to_select="auto", 
                                         direction="backward")
    backward.fit(X_train, y_train)
    backward_cols = list(X_train.columns[backward.get_support()])
    model_backward = sm.OLS(y_train, X_train[backward_cols]).fit()
        
    return model_insample, model_val, model_forward, forward_cols, model_backward, backward_cols


def getReport(X, y_trans, tran_name, lam=None):
    
    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_trans, test_size=0.2, random_state=31)
    
    model_insample, model_val, model_forward, forward_cols, model_backward, backward_cols = fit(X, y_trans, X_train, y_train)
    
    results = {
        "In-Sample": getQoF(model_insample, X, y_trans, tran_name, lam),
        "Validation": getQoF(model_val, X_test, y_test, tran_name, lam),
        "Forward": getQoF(model_forward, X_test[forward_cols], y_test, tran_name, lam),
        "Backward": getQoF(model_backward, X_test[backward_cols], y_test, tran_name, lam)
    }
    return pd.DataFrame(results)

# Auto MPG Transformed Regression

### Data Loading and Preprocessing

In [4]:
auto_mpg = pd.read_csv("dt/auto-mpg.csv",na_values="?", on_bad_lines="skip").dropna()
auto_mpg.rename(columns={"model year": "model_year"}, inplace=True)
auto_mpg.drop(columns=['car name'], inplace=True)
auto_mpg["mpg"] = auto_mpg.pop("mpg")
auto_mpg = sm.add_constant(auto_mpg)

auto_mpg['mpg_sqrt']  = np.sqrt(auto_mpg['mpg'])
auto_mpg['mpg_log1p'] = np.log1p(auto_mpg['mpg'])
auto_mpg['mpg_boxcox'] = boxcox(auto_mpg['mpg'], 0.19)

X = auto_mpg.iloc[:,:7]
y_orig = auto_mpg["mpg"]
y_sqrt = auto_mpg["mpg_sqrt"]
y_log1p = auto_mpg["mpg_log1p"]
y_boxcox = auto_mpg["mpg_boxcox"]

X.head()

,const,cylinders,displacement,horsepower,weight,acceleration,model_year
0,1.000,8,307.000,130.000,3504,12.000,70
1,1.000,8,350.000,165.000,3693,11.500,70
2,1.000,8,318.000,150.000,3436,11.000,70
3,1.000,8,304.000,150.000,3433,12.000,70
4,1.000,8,302.000,140.000,3449,10.500,70


### Auto MPG Regression: Square-Root, Log1p, and Box–Cox Transformations

In [5]:
df_sqrt   = getReport(X, y_sqrt, "sqrt")
df_log1p  = getReport(X, y_log1p, "log1p")
df_boxcox = getReport(X, y_boxcox, "boxcox", lam=0.19)

In [6]:
df_sqrt.to_latex("latex/df_sqrt_autoMPG.tex", float_format="%.3f")
df_log1p.to_latex("latex/df_log1p_autoMPG.tex", float_format="%.3f")
df_boxcox.to_latex("latex/df_boxcox_autoMPG.tex", float_format="%.3f")

In [7]:
print("Auto MPG Regression with Square-Root Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_sqrt)

Auto MPG Regression with Square-Root Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

        In-Sample  Validation  Forward  Backward
rSq         0.835       0.822    0.823     0.822
rSqBar      0.833       0.807    0.818     0.814
sst     23818.993    5372.801 5372.801  5372.801
sse      3926.849     955.390  950.390   958.806
mse         4.017       6.094   10.030     9.137
rmse        3.165       3.478    3.468     3.484
mae         2.347       2.376    2.375     2.350
m         392.000      79.000   79.000    79.000
dfr         6.000       6.000    2.000     3.000
df        385.000      72.000   76.000    75.000
fStat     825.235     120.822  220.454   161.034


In [8]:
print("Auto MPG Regression with Log1p Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_log1p)

Auto MPG Regression with Log1p Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

        In-Sample  Validation  Forward  Backward
rSq         0.849       0.832    0.806     0.804
rSqBar      0.847       0.818    0.801     0.794
sst     23818.993    5372.801 5372.801  5372.801
sse      3594.229     900.453 1044.122  1052.182
mse         3.169       5.398   11.217     9.319
rmse        3.028       3.376    3.635     3.649
mae         2.184       2.221    2.710     2.713
m         392.000      79.000   79.000    79.000
dfr         6.000       6.000    2.000     4.000
df        385.000      72.000   76.000    74.000
fStat    1063.694     138.083  192.956   115.912


In [9]:
print("Auto MPG Regression with Box-Cox Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_boxcox)

Auto MPG Regression with Box-Cox Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

        In-Sample  Validation  Forward  Backward
rSq         0.845       0.830    0.829     0.829
rSqBar      0.843       0.816    0.825     0.823
sst     23818.993    5372.801 5372.801  5372.801
sse      3683.218     914.477  917.735   916.540
mse         3.396       5.576    9.617     8.602
rmse        3.065       3.402    3.408     3.406
mae         2.229       2.262    2.356     2.234
m         392.000      79.000   79.000    79.000
dfr         6.000       6.000    2.000     3.000
df        385.000      72.000   76.000    75.000
fStat     988.221     133.268  231.627   172.688


# Boston House Price Transformed Regression

### Data Loading and Preprocessing

In [10]:
house_price = pd.read_csv("boston_house_prices.csv").dropna()
house_price['House_Price_Sqrt'] = np.sqrt(house_price['House_Price'])
house_price['House_Price_Log1p'] = np.log1p(house_price['House_Price'])
house_price['House_Price_Boxcox'] = boxcox(house_price['House_Price'], 0.85)
house_price = sm.add_constant(house_price)

X = house_price.iloc[:,:8]
y_orig = house_price["House_Price"]
y_sqrt = house_price["House_Price_Sqrt"]
y_log1p = house_price["House_Price_Log1p"]
y_boxcox = house_price["House_Price_Boxcox"]

X.head()

,const,Square_Footage,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size,Garage_Size,Neighborhood_Quality
0,1.000,1360,2,1,1981,0.600,0,5
1,1.000,4272,3,3,2016,4.753,1,6
2,1.000,3592,1,2,2016,3.635,0,9
3,1.000,966,1,2,1977,2.731,1,8
4,1.000,4926,2,1,1993,4.699,0,8


### Boston House Price Regression: Square-Root, Log1p, and Box–Cox Transformations

In [11]:
df_sqrt   = getReport(X, y_sqrt, "sqrt")
df_log1p  = getReport(X, y_log1p, "log1p")
df_boxcox = getReport(X, y_boxcox, "boxcox", lam=0.85)

In [12]:
df_sqrt.to_latex("latex/df_sqrt_housePrice.tex", float_format="%.3f")
df_log1p.to_latex("latex/df_log1p_housePrice.tex", float_format="%.3f")
df_boxcox.to_latex("latex/df_boxcox_housePrice.tex", float_format="%.3f")

In [24]:
print("Boston House Price Regression with Square-Root Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_sqrt)

Boston House Price Regression with Square-Root Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation         Forward        Backward
rSq               0.753           0.706           0.702           0.704
rSqBar            0.751           0.697           0.699           0.699
sst    196074221568.367 35797001122.000 35797001122.000 35797001122.000
sse     48497528289.829 10509483431.295 10649651869.890 10582381967.672
mse        36246276.223    39214482.415    39737503.977    39486494.879
rmse           6020.489        6262.147        6303.769        6283.828
mae            3613.896        3611.073        3687.410        3606.738
m              1338.000         268.000         268.000         268.000
dfr               8.000           8.000           3.000           5.000
df             1329.000         259.000         264.000         262.000
fStat           508.937          80.606         210.946         127.71

In [23]:
print("Boston House Price Regression with Log1p Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_log1p)

Boston House Price Regression with Log1p Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation            Forward          Backward
rSq               0.523           0.505           -415.056           -53.303
rSqBar            0.520           0.490           -419.784           -54.340
sst    196074221568.367 35797001122.000    35797001122.000   35797001122.000
sse     93570168847.798 17725546380.471 14893568669580.479 1943894999500.171
mse        69932853.620    66140090.435    55573017420.808    7253339545.374
rmse           8362.587        8132.656         235739.300         85166.540
mae            4219.512        4231.425          66672.183         25327.152
m              1338.000         268.000            268.000           268.000
dfr               8.000           8.000              3.000             5.000
df             1329.000         259.000            264.000           262.000
fStat           183.2

In [22]:
print("Boston House Price Regression with Box-Cox Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_boxcox)

Boston House Price Regression with Box-Cox Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation           Forward         Backward
rSq               0.576           0.546           -61.221          -10.125
rSqBar            0.574           0.532           -61.928          -10.337
sst    196074221568.367 35797001122.000   35797001122.000  35797001122.000
sse     83083467025.754 16248889297.412 2227326392657.103 398225509782.019
mse        62095258.835    60630175.946    8310919372.586   1485916076.276
rmse           7880.055        7786.539         91164.244        38547.582
mae            4052.895        4083.531         31806.787        14527.915
m              1338.000         268.000           268.000          268.000
dfr               8.000           8.000             3.000            5.000
df             1329.000         259.000           264.000          262.000
fStat           227.454          40.302  

# Medical Cost Transformed Regression

### Data Loading and Preprocessing

In [16]:
insurance = pd.read_csv("insurance_cat2num.csv").dropna()
insurance['charges_sqrt'] = np.sqrt(insurance['charges'])
insurance['charges_log1p'] = np.log1p(insurance['charges'])
insurance['charges_boxcox'] = boxcox(insurance['charges'], 0.04)

X = insurance.iloc[:,:9]
y_orig = insurance["charges"]
y_sqrt = insurance["charges_sqrt"]
y_log1p = insurance["charges_log1p"]
y_boxcox = insurance["charges_boxcox"]

X.head()

,intercept,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,1,19,27.900,0,0,1,0,0,1
1,1,18,33.770,1,1,0,0,1,0
2,1,28,33.000,3,1,0,0,1,0
3,1,33,22.705,0,1,0,1,0,0
4,1,32,28.880,0,1,0,1,0,0


### Medical Cost Regression: Square-Root, Log1p, and Box–Cox Transformations

In [17]:
df_sqrt   = getReport(X, y_sqrt, "sqrt")
df_log1p  = getReport(X, y_log1p, "log1p")
df_boxcox = getReport(X, y_boxcox, "boxcox", lam=0.04)

In [18]:
df_sqrt.to_latex("latex/df_sqrt_medicalCost.tex", float_format="%.3f")
df_log1p.to_latex("latex/df_log1p_medicalCost.tex", float_format="%.3f")
df_boxcox.to_latex("latex/df_boxcox_medicalCost.tex", float_format="%.3f")

In [19]:
print("Medical Cost  Regression with Square-Root Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_sqrt)

Medical Cost  Regression with Square-Root Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation         Forward        Backward
rSq               0.753           0.706           0.702           0.704
rSqBar            0.751           0.697           0.699           0.699
sst    196074221568.367 35797001122.000 35797001122.000 35797001122.000
sse     48497528289.829 10509483431.295 10649651869.890 10582381967.672
mse        36246276.223    39214482.415    39737503.977    39486494.879
rmse           6020.489        6262.147        6303.769        6283.828
mae            3613.896        3611.073        3687.410        3606.738
m              1338.000         268.000         268.000         268.000
dfr               8.000           8.000           3.000           5.000
df             1329.000         259.000         264.000         262.000
fStat           508.937          80.606         210.946         127.713


In [20]:
print("Medical Cost Regression with Log1p Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_log1p)

Medical Cost Regression with Log1p Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation            Forward          Backward
rSq               0.523           0.505           -415.056           -53.303
rSqBar            0.520           0.490           -419.784           -54.340
sst    196074221568.367 35797001122.000    35797001122.000   35797001122.000
sse     93570168847.798 17725546380.471 14893568669580.479 1943894999500.171
mse        69932853.620    66140090.435    55573017420.808    7253339545.374
rmse           8362.587        8132.656         235739.300         85166.540
mae            4219.512        4231.425          66672.183         25327.152
m              1338.000         268.000            268.000           268.000
dfr               8.000           8.000              3.000             5.000
df             1329.000         259.000            264.000           262.000
fStat           183.219    

In [21]:
print("Medical Cost Regression with Box-Cox Transformation:")
print("In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)\n")

print(df_boxcox)

Medical Cost Regression with Box-Cox Transformation:
In-Sample Analysis, Validation, and Feature Selection (Forward, Backward)

              In-Sample      Validation           Forward         Backward
rSq               0.576           0.546           -61.221          -10.125
rSqBar            0.574           0.532           -61.928          -10.337
sst    196074221568.367 35797001122.000   35797001122.000  35797001122.000
sse     83083467025.754 16248889297.412 2227326392657.103 398225509782.019
mse        62095258.835    60630175.946    8310919372.586   1485916076.276
rmse           7880.055        7786.539         91164.244        38547.582
mae            4052.895        4083.531         31806.787        14527.915
m              1338.000         268.000           268.000          268.000
dfr               8.000           8.000             3.000            5.000
df             1329.000         259.000           264.000          262.000
fStat           227.454          40.302        